# Instructions

1. Click the button "Not Trusted" on the menu bar of this notebook (at the top-right), and change the value to "Trusted".
2. Click Cell -> Run All. 
3. Read the documentation and understand the framework. 
4. Write Python code to answer the following questions. 
5. Please remember to save your notebook before submitting it on Compass2g.

## Notes

* Please use Python 3.3 or above. We can make no guarantees that the framework code will work for older versions of Python.
* Your algorithms will be tested on a set of different relations, and you may receive partial credit based on the number of test cases passed. You may also receive partial credit if your algorithm is correct, but the IO cost is higher than expected. 
* Importing libraries beyond what has already been imported is ***forbidden***. Any answer that contains an import statement will be given 0 points.

# Documentation

In this MP, you will use a framework that simulates the main memory and secondary memory of a DBMS. Within this framework, you will use Python code to define query processing algorithms, while heeding certain constraints:

 * The variables that your algorithm needs to work with are contained in Blocks, which are laid out on either main memory or disk. 
 * A Block must be in main memory before its contents can be read. 
 * The maximum number of blocks that can be stored in main memory is limited compared to the total amount of disk space. 
 * Every time a Block is moved in between main memory and disk, the total cost incurred by your algorithm increases. 
 
### Organization

 * An Attribute is the basic unit of storage in the framework. Attributes represent a string or an integer. An Attribute can only be operated on when it has been allocated in memory. The value of an Attribute can not be directly accessed. 
 * A Tuple (i.e., a record) can store up to 4 Attributes. 
 * A Block can store up to 5 tuples. 
 * A Buffer is a list of Blocks with variable length. Every Block must be contained within a Buffer. Buffers represent base relations and temporary relations. 
 * Main memory and disk can store an arbitrary number of Buffers. However, the total number of Blocks they can store is limited. At most 10 blocks can be stored in memory, and at most 100 blocks can be stored on disk. 
 
### Interface

All objects have the method `show()`, which displays an HTML representation of the object. 

**System**
 * `system.clear()` - remove all objects from both main memory and disk. 
 * `system.get_io_cost()` - returns the number of IO operations that your algorithm has used so far. 

**Memory / Disk**
 * `memory.size()` / `disk.size()` - return the number of blocks currently used by memory / disk, as an integer. 
 * `memory.clear()` / `disk.clear()` - remove all buffers and blocks from this storage object. Resets the size of this storage object to 0. 
 * `memory.new_buffer(size = 0, name = None)` / `disk.new_buffer(size = 0, name = None)` - create and return a new empty buffer with the given name. If `size` is greater than 0, then the buffer will be initialized with a number of empty blocks equal to `size`. 
 * `memory.remove_buffer(buffer)` / `disk.remove_buffer(buffer)` - remove a buffer and all blocks contained in the buffer from memory / disk. 
 
**Attribute**
 * `attribute.set_value(value)` - if `value` is a constant, then `attribute` is set to `value`. If `value` is another Attribute, then the contents of `attribute` are overwritten with the contents of `value`. 
 * `attribute`s behave like Python strings / integers, and the following operations can be applied on them: `==`, `!=`, `>=`, `>`, `<=`, `<`, `+`, `-`, `*`, `/`, `//`, `+=`, `-=`, `*=`, `/=`, `//=`. 

**Tuple**
 * `tuple.append_attr(attr)` - add an Attribute to the end of this tuple, if this tuple does not contain the max number of attributes. 
 * `tuple.combine(other)` - joins the attributes of two tuples. If `tuple` contains (a, b) and `other` contains (c, d), then `tuple.combine(other)` will contain (a, b, c, d). 

**Block**
 * `block.append_tuple(tuple)` - add a tuple to the end of this block. If `tuple` is a member of the Tuple class, then the tuple will be copied to the end of this block. If `tuple` is a constant, then a new tuple object will be added to the end of this block. 
 * `block.is_empty()` - return True if this block contains 0 tuples, and False otherwise. 
 * `block.is_full()` - return True if the number of tuples in this block is equal to the max capacity (5), and False otherwise. 
 * Block is a subclass of the native list class, so some list functions (ex: `block.pop()`, `iter(block)`, `len(block)`) will work. 

**Buffer**
 * `buffer.new_empty_block()` - add a new block with 0 tuples to the end of this buffer. Increases the size of the buffer by 1. 
 * `buffer.append_block(block)` - add a copy of the Block to the end of this buffer. Increases the size of the buffer by 1. 
 * `buffer.insert_copy_of_block(index, block)` - insert a copy of the Block into a specified position in the buffer. Increases the size of the buffer by 1. 
 * `buffer.write_to(other, start = 0, times = -1)` - overwrite the contents of `other` with the blocks in this buffer. If `start` is specified, then only the blocks after the `start` index are written. If `times` is specified, then at most `times` blocks are written to the `other` buffer. 
 * `buffer.iterate_over_all_tuples()` - returns an iterator ranging over all tuples in this buffer. 
    * Example usage: `for tup in buffer.iterate_over_all_tuples(): block.append_tuple(tup)`
 * `buffer.size()` - return the number of blocks stored in this buffer. 
 * Buffer is a subclass of the native list class, so some list functions (ex: `buffer.pop()`, `iter(buffer)`, `len(buffer)`) will work. 

**Index**
 * `index.lookup(value)` - return a list of (Block, tuple index) pairs that represent locations of tuples on disk where the indexed attribute matches the given `value`. 
 * `index.lookup_matching_blocks(value)` - return a list of unique blocks that contain tuples where the indexed attribute matches the given `value`. 

In [ ]:
# framework for MP 4

from math import floor, ceil
from system import *

### Question 1 - One-pass selection [10 pts]:

In the code cell below we have provided an implementation of the one-pass projection algorithm. Read the code carefully and use the `slideshow` debugging tool to examine the flow of the code in greater detail. Then, implement the one-pass selection algorithm in the `single_attr_equality_selection_one_pass` function. When you are done, press Shift-Enter to test your algorithm. 

Useful code fragments:
 * `tuple[attr]` - return an attribute in the tuple, indexed by the integer `attr`. For example, if `tuple` contains (0, 'a', 2), then `tuple[1]` will return 'a'. 

In [ ]:
def projection_one_pass(R, attrs):
    """Implement the SQL query:
       
       SELECT attr
       FROM R
       
       Params: R - input relation on disk, can have any size. 
               attrs - list of attributes to project. 
       Return: output - output relation on disk.
    """
    memory.clear()
    input_buffer = memory.new_buffer(name = "Input buffer")
    output = disk.new_buffer(name = "Output buffer")
    output_block = memory.new_buffer(name = "Buffer for output block").new_empty_block()
    slideshow.record("Start of one-pass projection")
    
    # iterate over all blocks in the relation
    for i, block in enumerate(R):
        # read a single block from the relation, into the input buffer in main memory
        input_buffer.clear()
        input_buffer.append_block(block)
        slideshow.record("Read in block #" + str(i) + " of R")
        
        # access the block in the input buffer
        input_block = input_buffer[0]
        
        # iterate over all tuples in the block
        for tup in input_block:
            # add projected attributes of tuple to output block
            projected_tuple = tup[attrs]
            output_block.append_tuple(projected_tuple)
            slideshow.record("Added tuple " + str(projected_tuple) + " to output block")
            
            # write output block to disk if block is full
            if output_block.is_full():
                output.append_block(output_block)
                output_block.clear()
                slideshow.record("Output block full; writing to disk")
                
    # write the rest of the output block to disk
    if not output_block.is_empty():
        output.append_block(output_block)
        slideshow.record("Writing rest of output block to disk")

    slideshow.record("End of one-pass projection")
    return output
    
def test_projection_one_pass():
    system.clear()
    r = [(0, 0, 0), (0, 1, 2), (3, 3, 3), (4, 6, 8), (5, 7, 9), (9, 8, 7)]
    R = disk.add_clustered_relation(r, name = "R")
    output = projection_one_pass(R, attrs = [0])

slideshow.clear()
test_projection_one_pass()
slideshow.show()

In [ ]:
def single_attr_equality_selection_one_pass(R, attr, val):
    """Implement the SQL query:
       
       SELECT *
       FROM R
       WHERE R.attr = val
       
       Params: R - input relation on disk, can have any size. 
               attr - attribute to compare value to. Must be an integer < number of attributes in tuples of R. 
               val - value to compare the attribute to. 
       Return: output - output relation on disk. 
    """
    memory.clear()
    input_buffer = memory.new_buffer(name = "Input buffer")
    output = disk.new_buffer(name = "Output buffer")
    output_block = memory.new_buffer(name = "Buffer for output block").new_empty_block()
    slideshow.record("Start of one-pass selection")
    
    # write your answer to Q1 here
    
    
    slideshow.record("End of one-pass selection")
    return output

def test_selection_one_pass():
    system.clear()
    r = [(0, 0), (0, 1), (3, 3), (4, 6), (5, 5), (5, 7), (9, 8), (7, 5), (5, 0)]
    R = disk.add_clustered_relation(r, name = "R")
    system.reset_io_cost(passcode = "")
    output = single_attr_equality_selection_one_pass(R, attr = 0, val = 5)
    print("Actual output:")
    output.show()
    print("Actual IO cost: " + str(system.get_io_cost()) + "\n")
    print("Expected output (order of tuples does not matter):")
    memory.clear()
    r_ = [(5, 5), (5, 7), (5, 0)]
    R_ = disk.add_clustered_relation(r_)
    R_.show()
    print("Expected IO cost: <= 2B(R) = 4")
    
slideshow.clear()
test_selection_one_pass()
slideshow.show()

### Question 2 - Block based nested loop join [20 pts]:

In the code cell below, we have provided an implementation of the tuple-based nested loop join. Tuple-based nested loop join only requires 3 blocks of main memory to be available, but it is less efficient than block-based nested loop join. In the function `block_based_nested_loop_join`, implement the block-based join algorithm. The IO cost of your algorithm should be less than the IO cost of the tuple-based join. 

Useful code fragments:
 * `buffer.write_to(other, start, times)`
 * `buffer.iterate_over_all_tuples()`
 * `r_tuple[r_join_attr] == s_tuple[s_join_attr]`
 * `r_tuple.combine(s_tuple)`

In [ ]:
def tuple_based_nested_loop_join(R, S, r_join_attr, s_join_attr):
    """Implement the SQL query:
    
       SELECT r, s
       FROM R, S
       WHERE r_join_attr == s_join_attr
       
       Params: R - input relation on disk, can have any size. 
               S - input relation on disk, can have any size. 
               r_join_attr - attribute of R to join on. Must be an integer < number of attributes in tuples of R. 
               s_join_attr - attribute of S to join on. Must be an integer < number of attributes in tuples of S. 
       Return: output - output relation on disk. 
       Note: B(S) < B(R)
    """
    memory.clear()
    r_input_buffer = memory.new_buffer(name = "R input buffer")
    s_input_buffer = memory.new_buffer(name = "S input buffer")
    output_block = memory.new_buffer(name = "Buffer for output block").new_empty_block()
    output = disk.new_buffer(name = "Output buffer")
    slideshow.record("Start of tuple-based nested loop join")
    
    for i, r_block in enumerate(R):
        r_input_buffer.clear()
        r_input_buffer.append_block(r_block)
        slideshow.record("Read block #" + str(i) + " from R")
        
        for j, s_block in enumerate(S):
            s_input_buffer.clear()
            s_input_buffer.append_block(s_block)
            slideshow.record("Read block #" + str(j) + " from S")

            for r_tuple in r_input_buffer[0]:
                for s_tuple in s_input_buffer[0]:
                    if r_tuple[r_join_attr] == s_tuple[s_join_attr]:
                        joined_tuple = r_tuple.combine(s_tuple)
                        output_block.append_tuple(joined_tuple)
                        slideshow.record("Found two tuples to join: " + str(r_tuple) + " and " + str(s_tuple))

                        if output_block.is_full():
                            output.append_block(output_block)
                            output_block.clear()
                            slideshow.record("Output block full; write to disk")

    if not output_block.is_empty():
        output.append_block(output_block)
        slideshow.record("Write rest of output block to disk")

    slideshow.record("End of tuple-based nested loop join")
    return output

def test_tuple_based_nested_loop_join():
    system.clear()
    r = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    R = disk.add_clustered_relation(r, name = "R")
    s = [(0, 0), (22, 2), (33, 3), (33, 3), (66, 6), (77, 7), (123, 9), (10, 100)]
    S = disk.add_clustered_relation(s, name = "S")
    
    system.reset_io_cost(passcode = "")
    output = tuple_based_nested_loop_join(R, S, 0, 1)
    print("IO cost of tuple-based join: " + str(system.get_io_cost()))
    
slideshow.clear()
test_tuple_based_nested_loop_join()
slideshow.show()

In [ ]:
def block_based_nested_loop_join(R, S, r_join_attr, s_join_attr):
    """Implement the SQL query:
    
       SELECT r, s
       FROM R, S
       WHERE r_join_attr == s_join_attr
       
       Params: R - input relation on disk, can have any size. 
               S - input relation on disk, can have any size. 
               r_join_attr - attribute of R to join on. Must be an integer < number of attributes in tuples of R. 
               s_join_attr - attribute of S to join on. Must be an integer < number of attributes in tuples of S. 
       Return: output - output relation on disk. 
       Note: B(S) < B(R)
    """
    memory.clear()
    r_input_buffer = memory.new_buffer(name = "R input buffer")
    s_input_buffer = memory.new_buffer(name = "S input buffer")
    output_block = memory.new_buffer(name = "Buffer for output block").new_empty_block()
    output = disk.new_buffer(name = "Output buffer")
    slideshow.record("Start of block-based nested loop join")

    # write your answer to Q2 here
    

    slideshow.record("End of block-based nested loop join")
    return output

def test_block_based_nested_loop_join():
    system.clear()
    r = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    R = disk.add_clustered_relation(r, name = "R")
    s = [(0, 0), (22, 2), (33, 3), (33, 3), (66, 6), (77, 7), (123, 9), (10, 100)]
    S = disk.add_clustered_relation(s, name = "S")
    system.reset_io_cost(passcode = "")
    output = block_based_nested_loop_join(R, S, 0, 1)
    print("Actual output:")
    output.show()
    print("Actual IO cost: " + str(system.get_io_cost()))
    print()
    print("Expected output (order of tuples does not matter):")
    print("Expected IO cost: <= 6")
    memory.clear()
    r_ = [(2, 22, 2), (3, 33, 3), (3, 33, 3), (6, 66, 6), (7, 77, 7), (9, 123, 9)]
    R_ = disk.add_clustered_relation(r_)
    R_.show()
    
slideshow.clear()
test_block_based_nested_loop_join()
slideshow.show()

### Question 3 - Multiway merge sort [20 pts]:

Broadly speaking, there are two parts to the multiway merge sort algorithm. The first part splits the input relation into sorted sublists, each of which is at most $M - 1$ blocks in size. The second part reads tuples from the front of the sorted sublists, one at a time, and writes the smallest tuple to the output buffer. In the code cell below, we have implemented the first part of the multiway merge sort algorithm. Your task is to implement the second part in the function `multiway_merge_sort`. When you are finished, run the `test_multiway_merge_sort` function to test your implementation. 

Useful code fragments:
 * `block.pop(tuple_index)` - remove the tuple at the location, `block[tuple_index]`. 
 * `buffer.pop(block_index)` - remove the block at the location, `buffer[block_index]`. 

In [ ]:
def _create_sorted_sublists(R):
    """Create B(R) / (M - 1) sorted sublists on disk, each M - 1 in size.
    """
    memory.clear()
    buffer = memory.new_buffer(name = "Input buffer for sorted sublist")
    swap_block = memory.new_buffer(name = "Buffer for swap block") \
                 .new_empty_block()
    slideshow.record("Creating sorted sublists for " + str(R.name))
    
    sublists = []
    for x, i in enumerate(range(0, len(R), M - 1)):
        buffer.clear()
        for j in range(i, min(i + M - 1, len(R))):
            buffer.append_block(R[j])
        slideshow.record("Reading chunk of blocks from table")

        # use bubble sort to sort the sublist
        did_swap = True
        while did_swap:
            did_swap = False
            tuple_indices = [(j, k)
                             for j, block in enumerate(buffer)
                             for k, tup in enumerate(block)]
            for (j1, k1), (j2, k2) in zip(tuple_indices, tuple_indices[1:]):
                if buffer[j1][k1] > buffer[j2][k2]:
                    # swap the two tuples if they are out of order
                    swap_block.append_tuple(buffer[j1][k1])
                    swap_block.append_tuple(buffer[j2][k2])
                    buffer[j1].pop(k1)
                    buffer[j1].insert_copy_of_tuple(k1, swap_block[1])
                    buffer[j2].pop(k2)
                    buffer[j2].insert_copy_of_tuple(k2, swap_block[0])
                    swap_block.clear()

                    did_swap = True

        slideshow.record("Sorting the chunk of blocks")
        
        sublist = disk.new_buffer(name = str(R.name) + " sublist #" + str(x))
        buffer.write_to(sublist)
        sublists.append(sublist)
        slideshow.record("Writing the sorted sublist to disk")
    
    memory.remove_buffer(buffer)
    slideshow.record("Finished creating sorted sublists for " + str(R.name))
    return sublists

def multiway_merge_sort(R):
    """Implement the SQL query:
    
       SELECT *
       FROM R
       SORT BY * ASC

       Note: it is assumed that M < B(R) < M^2 (approximately)
    """
    slideshow.record("Start of multiway merge sort")
    
    # phase 1: create B(R) / (M - 1) sorted sublists on disk, each M - 1 in size
    sublists = _create_sorted_sublists(R)

    # phase 2: merge the sorted sublists
    output = disk.new_buffer(name = "Output buffer")
    output_block = memory.new_buffer(name = "Buffer for output block").new_empty_block()
    
    # write your answer to Q3 here
    

    slideshow.record("End of multiway merge sort")
    return output

def test_multiway_merge_sort():
    system.clear()
    r = [0, 9, 4, 3, 2, 6, 2, 7, 8, 2, 5, 3,
         2, 6, 2, 1, 7, 4, 2, 5, 1, 4, 9, 6,
         5, 3, 7, 7, 4, 2, 8, 1, 3, 0, 6, 9,
         1, 7, 3, 2, 8, 0, 7, 6, 3, 7, 2, 8,
         7, 1, 2, 4, 8, 2, 3, 9, 8, 6, 0, 5,
         6, 4, 8, 1, 4, 2, 8, 1, 4, 7, 3, 8]
    n_blocks = ceil(len(r) / Block.TUPLES_PER_BLOCK)
    R = disk.add_clustered_relation(r, name = "R")
    system.reset_io_cost(passcode = "")
    output = multiway_merge_sort(R)
    merge_sort_cost = system.get_io_cost()
    
    memory.clear()
    buffer = memory.new_buffer()
    indices = [(i, j) for i, block in enumerate(output) for j in range(len(block))]
    for (b_i, t_i), (b_j, t_j) in zip(indices[: -1], indices[1 :]):
        buffer.clear()
        buffer.append_block(output[b_i])
        buffer.append_block(output[b_j])
        assert buffer[0][t_i] <= buffer[1][t_j], \
               "Output is not sorted in ascending order"
    assert len(output) == n_blocks, "Number of blocks in output is incorrect"
    print("Number of blocks in output is correct and tuples are sorted in ascending order")
    print("Actual IO cost: " + str(merge_sort_cost))
    print("Expected IO cost: <= 4B(R) = 60")
    
slideshow.clear()
test_multiway_merge_sort()
slideshow.show()

### Question 4 - Hash based duplicate elimination [20 pts]:

The first step in all hash based algorithms is to divide the relation into separate buffers, each of which fits in main memory, based on the hash values of its tuples. Your task is to implement this step in the function `hash_based_partition` below. When you are done, run the code cell to test the hash based duplicate elimination function, which relies on your implementation. 

Useful code fragments:
 * `hash(tup) % (M - 1)`

In [ ]:
def hash_based_partition(R):
    """Partition a relation into multiple buckets on disk, based on hash value.
       Returns a list of pointers to buffers on disk.
    """
    # create M - 1 buffers on memory, and M - 1 corresponding buckets on disk
    memory.clear()
    buckets = [disk.new_buffer(name = str(R.name) + " bucket #" + str(i)) for i in range(M - 1)]
    blocks = [memory.new_buffer(name = "Buffer #" + str(i)).new_empty_block() for i in range(M - 1)]
    input_buffer = memory.new_buffer(name = "Input buffer")
    slideshow.record("Start of hash-based partition for " + str(R.name))
    
    # write your answer to Q4 here
    
    
    return buckets

def hash_based_duplicate_elimination(R):
    """SELECT DISTINCT(*)
       FROM R

       Note: it is assumed that M < B(R) < M^2 (approximately)
    """
    slideshow.record("Start of hash based duplicate elimination")
    buckets = hash_based_partition(R)
    
    output = disk.new_buffer(name = "Output buffer")
    output_block = memory.new_buffer(name = "Buffer for output block").new_empty_block()
    for i, bucket in enumerate(buckets):
        bucket = bucket.move_to(memory)
        slideshow.record("Working with bucket " + str(i))

        last_min = None
        while True:
            cur_min = None
            for tup in bucket.iterate_over_all_tuples():
                if last_min is None or tup > last_min:
                    if cur_min is None or tup < cur_min:
                        cur_min = tup

            if cur_min is None:
                break

            last_min = cur_min

            output_block.append_tuple(cur_min)
            slideshow.record("Adding distinct tuple " + str(cur_min) + " to output block")
            if output_block.is_full():
                output.append_block(output_block)
                output_block.clear()
                slideshow.record("Output block full; writing to disk")

        memory.remove_buffer(bucket)

    if not output_block.is_empty():
        output.append_block(output_block)
        slideshow.record("Writing rest of output block to disk")

    slideshow.record("End of hash based duplicate elimination")
    return output

def test_hash_based_duplicate_elimination():
    system.clear()
    r = [0, 9, 4, 3, 2, 6, 2, 7, 8, 2, 5, 3,
         2, 6, 2, 1, 7, 4, 2, 5, 1, 4, 9, 6,
         5, 3, 7, 7, 4, 2, 8, 1, 3, 0, 6, 9,
         1, 7, 3, 2, 8, 0, 7, 6, 3, 7, 2, 8,
         7, 1, 2, 4, 8, 2, 3, 9, 8, 6, 0, 5,
         6, 4, 8, 1, 4, 2, 8, 1, 4, 7, 3, 8]
    R = disk.add_clustered_relation(r, name = "R")
    system.reset_io_cost(passcode = "")
    
    output = hash_based_duplicate_elimination(R)
    print("Actual output:")
    output.show()
    print("Actual IO cost: " + str(system.get_io_cost()))
    print()
    print("Expected output (order of tuples does not matter):")
    memory.clear()
    r_ = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    R_ = disk.add_clustered_relation(r_)
    R_.show()
    print("Expected IO cost: <= 60")
    
slideshow.clear()
test_hash_based_duplicate_elimination()
slideshow.show()

### Question 5 - Index based selection [20 pts]:

In this question, we will see how an index can speed up the selection operator by avoiding the cost of pulling every block in the relation into memory. We have provided an Index object that maps from the index attribute to the locations of tuples on disk. Using this Index object, write a function that performs selection more efficiently than the table-scan method. Once you are done, run the code cell to test your function. 

Useful code fragments:
 * `for block in index.lookup_matching_blocks(val)`
 * `tup[attr] == val`

In [ ]:
def index_based_selection(R, index, attr, val):
    """SELECT *
       FROM R
       WHERE attr = v
       
       Note: index on R.attr is clustering. 
    """
    output = disk.new_buffer(name = "Output buffer")    
    input_buffer = memory.new_buffer(name = "Input buffer")
    output_block = memory.new_buffer(name = "Buffer for output block").new_empty_block()
    slideshow.record("Start of index based selection")

    # write your answer to Q5 here
    

    slideshow.record("End of index based selection")
    return output

def create_index(r, R, index_attr):
    """creates a single-dim index on a relation.
       r is the raw values of relation.
       assumes that Storage.add_clustered_relation is implemented in certain way
    """
    index = []
    i = 0
    for block in R:
        for tuple_ind, _ in enumerate(block):
            tup = r[i] if type(r[i]) is tuple else (r[i], )
            index.append((tup[index_attr], block, tuple_ind))
            i += 1
    return Index(index)

def test_index_based_selection():
    system.clear()
    r = [(0, 0), (0, 1), (0, 2), (1, 3), (1, 4), (1, 5), (1, 6),
         (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (2, 12), (3, 13),
         (3, 14), (4, 15), (5, 16), (5, 17), (5, 18), (6, 19)]
    R = disk.add_clustered_relation(r, name = "R")
    index = create_index(r, R, 0)
    system.reset_io_cost(passcode = "")

    output = index_based_selection(R, index, attr = 0, val = 2)

    print("Actual output:")
    output.show()
    print("Actual IO cost: " + str(system.get_io_cost()))
    print()
    print("Expected output (order of tuples does not matter):")
    memory.clear()
    r_ = [(2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (2, 12)]
    R_ = disk.add_clustered_relation(r_)
    R_.show()
    print("Expected IO cost: < 2B(R) = 8")
    
slideshow.clear()
test_index_based_selection()
slideshow.show()

### Question 6 - Query optimization [10 pts]:

In this question, we will build upon the algorithms that you have just written to execute a complete SQL query. More specifically, suppose that we have two relations, $R$ and $S$, that obey the following schema:

Now, we wish to execute the following SQL query using the query processing algorithms that you have written. 

The code cell below carries out an unoptimized query plan for executing the query. Using the algebraic laws of relational algebra and your common sense, can you come up with a more efficient query plan? Note that in order to receive full credit, your answers to Q1-5 must be functional, and the cost of your optimized query plan should have ~20% less cost than the unoptimized version. 

In [ ]:
def unoptimized_query(R, S, R_index, n):
    # equi-join R and S on the attributes R.b and S.x
    # this creates a relation with attributes (R.a, R.b, S.x, S.y)
    r_and_s = block_based_nested_loop_join(R, S, r_join_attr = 1, s_join_attr = 0)
    
    # select tuples such that R.a = n
    filtered_r_and_s = single_attr_equality_selection_one_pass(r_and_s, attr = 0, val = n)
    
    # perform duplicate elimination
    unique_r_and_s = hash_based_duplicate_elimination(filtered_r_and_s)
    
    # project out the attribute S.y
    # this creates a relation with attributes (S.y)
    select_s_y = projection_one_pass(unique_r_and_s, attrs = [3])
    
    # sort the relation on the attribute S.y
    sorted_s_y = multiway_merge_sort(select_s_y)
    
    # perform duplicate elimination
    unique_s_y = hash_based_duplicate_elimination(sorted_s_y)
    
    # re-sort the relation, and return the result
    return multiway_merge_sort(unique_s_y)

def optimized_query(R, S, R_index, n):
    # write your answer to Q6 here
    pass

def test_optimized_query():
    def test_query(func):
        system.clear()
        r = [(0, "ann"), (1, "bob"), (1, "cal"), (1, "dan"), (1, "ed"), (1, "fan"), (1, "gal"), 
             (1, "hal"), (1, "ian"), (1, "jeb"), (1, "ken"), (2, "leo"), (2, "mia"), (3, "nob"),
             (3, "oda"), (4, "pam"), (5, "quo"), (6, "ray"), (7, "stu"), (8, "tom"), (9, "ura"),
             (10, "ves"), (11, "wil"), (12, "xia"), (13, "yon"), (14, "zek")]
        R = disk.add_clustered_relation(r, name = "R")
        R_index = create_index(r, R, 0)
        
        s = [("ann", 0), ("bob", 1), ("cal", 2), ("dan", 3), ("ed", 4), ("fan", 0), ("gal", 0),
             ("huc", 5), ("ian", 7), ("ken", 9), ("leo", 12), ("mia", 9)]
        S = disk.add_clustered_relation(s, name = "S")
        
        system.reset_io_cost(passcode = "")
        return func(R, S, R_index, n = 1)
    
    optimized_ans = test_query(optimized_query)
    optimized_cost = system.get_io_cost()
    print("Optimized query output")
    optimized_ans.show()
    print("Optimized query cost: " + str(optimized_cost) + "\n")
    
    unoptimized_ans = test_query(unoptimized_query)
    unoptimized_cost = system.get_io_cost()
    print("Unoptimized query output")
    unoptimized_ans.show()
    print("Unoptimized query cost: " + str(unoptimized_cost) + "\n")
    
    print("Expected output (order of tuples does not matter):")
    memory.clear()
    r_ = [0, 1, 2, 3, 4, 7, 9]
    R_ = disk.add_clustered_relation(r_)
    R_.show()
    
test_optimized_query()